# Sourcing Federal Data: Higher Education Data
### A step-by-step guide to building three-year data panel from federal higher education data sources

Available online: [_Sourcing Federal Data: Higher Education Data: 
A step-by-step guide to building three-year data panel from federal higher education data sources_](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#links)


In [1]:
# STEP ONE
# ========

import requests
import zipfile
import io
import pandas as pd
# We use the ipeds prefix because the data comes from the:
# [I]ntegrated [P]ostsecondary [E]ducation [D]ata [S]ystem
ipeds_locs = 'https://nces.ed.gov/ipeds/datacenter/data/'
ipeds_fils = 'HD{}_Data_Stata.zip'
ipeds_dict = 'HD{}_Dict.zip'
years = [2015,2016,2017]

In [2]:
# STEPS TWO THROUGH FOUR
# ======================

# STEP TWO:
# =========
for yr in years:
    print('GETTING FILES FROM {}'.format(yr))
    rdata = requests.get(ipeds_locs + ipeds_fils.format(yr))
    rdict = requests.get(ipeds_locs + ipeds_dict.format(yr))
    rdata_zip = zipfile.ZipFile(io.BytesIO(rdata.content))
    rdict_zip = zipfile.ZipFile(io.BytesIO(rdict.content))
    
    print('Extracting {} files from zip archive:'.format(yr))
    rdata_zip.printdir()
    rdict_zip.printdir()
    rdata_zip.extractall()
    rdict_zip.extractall()
    
    print('Saving zip archive to disk.')
    open(ipeds_fils.format(yr), 'wb').write(rdata.content)
    open(ipeds_dict.format(yr), 'wb').write(rdict.content)
    
    
    # STEP THREE
    # ==========
    print('Replacing Code Values with Code Labels.')
    
    # Extract frequencies tab the data dictionary (hdYYYY.xlsx)
    freqs = pd.read_excel('hd{}.xlsx'.format(yr),
                          sheet_name='Frequencies')
    # Put institutional data into a data frame (df)
    df = pd.read_csv('hd{}_data_stata.csv'.format(yr), 
                     encoding='ISO-8859-1')    
    
    # Get list of categorical variable names
    cat_colms = set(freqs['varname'])
    
    # Remove fips code to prevent its modification
    cat_colms.remove('FIPS')
     
    # Loop through categorical columns
    for col in cat_colms:
        # Get map keys (code values)
        code_values = freqs[freqs['varname'] == col]['codevalue']
        # Convert map keys to int where appropriate
        code_values = [int(i) if str(i).isdigit() 
                       else i for i in code_values]
        # Get map value (ValueLabels)
        code_labels = freqs[freqs['varname'] == col]['valuelabel']
        var_map = dict(zip(code_values, code_labels)) 
        # Apply mapping dictionary to categorical column
        df[col] = df[col].map(var_map)
    
    
    # STEP FOUR
    # =========
    
    # Create time index for panel specification
    df['year'] = yr
    
    print('Writing hd{}_data_stata.csv as csv, pkl'.format(yr))
    df.columns = [i.lower() for i in df.columns]
    df.to_csv('hd{}_data_stata.csv'.format(yr))
    df.to_pickle('hd{}_data_stata.pkl'.format(yr))
    print('Done!', end='\n\n')

GETTING FILES FROM 2015
Extracting 2015 files from zip archive:
File Name                                             Modified             Size
hd2015_data_stata.csv                          2016-10-11 14:16:10      4155427
File Name                                             Modified             Size
hd2015.xlsx                                    2016-10-17 15:44:44       253309
Saving zip archive to disk.
Replacing Code Values with Code Labels.
Writing hd2015_data_stata.csv as csv, pkl
Done!

GETTING FILES FROM 2016
Extracting 2016 files from zip archive:
File Name                                             Modified             Size
hd2016_data_stata.csv                          2017-12-18 09:20:12      4552957
File Name                                             Modified             Size
hd2016.xlsx                                    2018-01-08 11:23:52       252085
Saving zip archive to disk.
Replacing Code Values with Code Labels.
Writing hd2016_data_stata.csv as csv, pkl
Done!

In [4]:
# STEP FIVE
# =========

all_data = {}
for yr in years:
    all_data[yr] = pd.read_csv('hd{}_data_stata.csv'.format(yr))
    
df = pd.concat(all_data).sort_values(['unitid',
                                      'year']).set_index(['unitid',
                                                          'year'])

In [5]:
# DISPLAY RESULTS - DISPLAY FIRST THREE INSTITUTIONS
# ==================================================

df[['instnm','city','stabbr','control','locale']].head(n=9)

instnm        city   stabbr  \
unitid year                                                             
100654 2015             Alabama A & M University      Normal  Alabama   
       2016             Alabama A & M University      Normal  Alabama   
       2017             Alabama A & M University      Normal  Alabama   
100663 2015  University of Alabama at Birmingham  Birmingham  Alabama   
       2016  University of Alabama at Birmingham  Birmingham  Alabama   
       2017  University of Alabama at Birmingham  Birmingham  Alabama   
100690 2015                   Amridge University  Montgomery  Alabama   
       2016                   Amridge University  Montgomery  Alabama   
       2017                   Amridge University  Montgomery  Alabama   

                            control         locale  
unitid year                                         
100654 2015                  Public  City: Midsize  
       2016                  Public  City: Midsize  
       2017                  Public  City: Midsize  
100663 2015                  Public  City: Midsize  
       2016                  Public  City: Midsize  
       2017                  Public  City: Midsize  
100690 2015  Private not-for-profit  City: Midsize  
       2016  Private not-for-profit  City: Midsize  
       2017  Private not-for-profit  City: Midsize

In [6]:
# DISPLAY RESULTS - DISPLAY A SPECIFIC INSTITUTION (By Name)
# ==========================================================

df[df['instnm'] == 'University of Wisconsin-Madison']

Unnamed: 0                           instnm            addr  \
unitid year                                                                
240444 2015        4263  University of Wisconsin-Madison  500 Lincoln Dr   
       2016        4150  University of Wisconsin-Madison  500 Lincoln Dr   
       2017        4029  University of Wisconsin-Madison  500 Lincoln Dr   

                city     stabbr         zip  fips                      obereg  \
unitid year                                                                     
240444 2015  Madison  Wisconsin  53706-1380    55  Great Lakes IL IN MI OH WI   
       2016  Madison  Wisconsin  53706-1380    55  Great Lakes IL IN MI OH WI   
       2017  Madison  Wisconsin  53706-1380    55  Great Lakes IL IN MI OH WI   

                     chfnm    chftitle  ...  \
unitid year                             ...   
240444 2015  Rebecca Blank  Chancellor  ...   
       2016  Rebecca Blank  Chancellor  ...   
       2017  Rebecca Blank  Chancellor  ...   

                                        f1syscod         countycd  \
unitid year                                                         
240444 2015  WI - University of Wisconsin System  Dane County, WI   
       2016  WI - University of Wisconsin System  Dane County, WI   
       2017  WI - University of Wisconsin System  Dane County, WI   

                countynm         cngdstcd    longitud   latitude  \
unitid year                                                        
240444 2015  Dane County  WI, District 02  -89.405356  43.073858   
       2016  Dane County  WI, District 02    -89.4054    43.0739   
       2017  Dane County  WI, District 02    -89.4041    43.0754   

                                                       dfrcgid  \
unitid year                                                      
240444 2015  Doctoral Universities: Highest Research Activi...   
       2016  Doctoral Universities: Highest Research Activi...   
       2017  Doctoral Universities: Highest Research Activi...   

                                                      dfrcuscg       duns  \
unitid year                                                                 
240444 2015  Yes, institution submitted a custom comparison...        NaN   
       2016  Yes, institution submitted a custom comparison...  170403497   
       2017  Yes, institution submitted a custom comparison...  189000854   

                                disaurl  
unitid year                              
240444 2015                         NaN  
       2016          mcburney.wisc.edu/  
       2017  https://mcburney.wisc.edu/  

[3 rows x 72 columns]

In [10]:
# DISPLAY RESULTS - DISPLAY A SPECIFIC INSTITUTION (By Unitid)
# ===========================================================

df.loc[174066]

,Unnamed: 0,instnm,addr,city,stabbr,zip,fips,obereg,chfnm,chftitle,...,f1syscod,countycd,countynm,cngdstcd,longitud,latitude,dfrcgid,dfrcuscg,duns,disaurl
year,,,,,,,,,,,,,,,,,,,,,
2015,1973,University of Minnesota-Twin Cities,100 Church Street SE,Minneapolis,Minnesota,55455-0213,27,Plains IA KS MN MO NE ND SD,Eric W. Kaler,President,...,MN - University of Minnesota,"Hennepin County, MN",Hennepin County,"MN, District 05",-93.235352,44.977886,Doctoral Universities: Highest Research Activi...,"Yes, institution submitted a custom comparison...",NaN,NaN
2016,1911,University of Minnesota-Twin Cities,100 Church Street SE,Minneapolis,Minnesota,55455-0213,27,Plains IA KS MN MO NE ND SD,Eric W. Kaler,President,...,MN - University of Minnesota,"Hennepin County, MN",Hennepin County,"MN, District 05",-93.2354,44.9779,Doctoral Universities: Highest Research Activi...,"Yes, institution submitted a custom comparison...",555917996,https://diversity.umn.edu/disability/
2017,1859,University of Minnesota-Twin Cities,100 Church Street SE,Minneapolis,Minnesota,55455-0213,27,Plains IA KS MN MO NE ND SD,Eric W. Kaler,President,...,MN - University of Minnesota,"Hennepin County, MN",Hennepin County,"MN, District 05",-93.2355,44.9729,Doctoral Universities: Highest Research Activi...,"Yes, institution submitted a custom comparison...",555917996,https://diversity.umn.edu/disability/
